In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import requests

In [2]:
densidad = pd.read_csv('../Data/clean_data/densidad_poblacion.csv')

In [3]:
densidad.head(1)

,prov_id,mun_id,total_2023,hombres_2023,mujeres_2023,ine_id,municipio,provincia,comunidad,km2,densidad
0,1,1001,2975.0,1539.0,1436.0,1001000000,Alegria-dulantzi,Provincia de Alaba,Pais Vasco,1995.0,1.491228


In [4]:
contagio_mun = densidad.loc[:,['densidad', 'prov_id', 'mun_id', 'municipio']]

In [5]:
contagio_mun.head(2)

,densidad,prov_id,mun_id,municipio
0,1.491228,1,1001,Alegria-dulantzi
1,1.070368,1,1002,Amurrio


In [6]:
#calculating zombie virus estimated basic reproduction number (transmision rate): infectiousnes * population density

c = 5 #constant representing infectiousness of the virus (For reference, flu is 1.3, measles is 12-18. We are picking 8 given that every human is susceptible of contracting the virus, there is no recovery and once bitten you are infected with a 100% probability.)

In [7]:
contagio_mun['R0'] = contagio_mun.densidad*c

In [8]:
contagio_mun['prob_inf'] = 1 - np.exp((-1*contagio_mun.R0*contagio_mun.densidad))

In [9]:
contagio_mun.head(3)

,densidad,prov_id,mun_id,municipio,R0,prob_inf
0,1.491228,1,1001,Alegria-dulantzi,7.456140,0.999985
1,1.070368,1,1002,Amurrio,5.351842,0.996748
2,0.192302,1,1003,Aramaio,0.961512,0.168814


In [20]:
#contagio_mun.to_csv('contagio_municipios.csv', encoding='utf-8-sig', index = False)

In [11]:
contagio_prov = densidad.loc[:,['prov_id', 'provincia']]

In [12]:
contagio_prov.drop_duplicates(inplace = True)

In [13]:
contagio_prov['km2'] = contagio_prov.prov_id.apply(lambda x: sum(densidad[densidad.prov_id == x]['km2']))

In [14]:
contagio_prov['total_prov'] = contagio_prov.prov_id.apply(lambda x: sum(densidad[densidad.prov_id == x]['total_2023']))

In [15]:
contagio_prov['densidad'] = contagio_prov['total_prov'] / contagio_prov['km2']
contagio_prov.head(2)

,prov_id,provincia,km2,total_prov,densidad
0,1,Provincia de Alaba,293470.00,336686.0,1.147259
51,2,Provincia de Albacete,1299348.99,387174.0,0.297975


In [16]:
contagio_prov['R0'] = contagio_prov.densidad*c
contagio_prov['prob_inf'] = 1 - np.exp((-1*contagio_prov.R0*contagio_prov.densidad))

In [17]:
contagio_prov.head(3)

,prov_id,provincia,km2,total_prov,densidad,R0,prob_inf
0,1,Provincia de Alaba,293470.00,336686.0,1.147259,5.736293,0.998614
51,2,Provincia de Albacete,1299348.99,387174.0,0.297975,1.489877,0.358500
123,2,Provincia de Alicante,1299348.99,387174.0,0.297975,1.489877,0.358500


In [19]:
#contagio_prov.to_csv('contagio_provincias.csv', encoding='utf-8-sig', index = False)

In [21]:
prov_codes = contagio_prov.loc[:,['prov_id','provincia']]

In [23]:
#prov_codes.to_csv('prov_codes.csv', encoding='utf-8-sig', index = False)